In [2]:
import pandas as pd

In [3]:
features = pd.read_csv(r'C:\project\features.csv', index_col='match_id')

In [4]:
features_test = pd.read_csv(r'C:\project\features_test.csv', index_col='match_id')

In [5]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [6]:
features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


# Подход 1: градиентный бустинг "в лоб"

удаляем ненужные столбци из "будущего"

In [4]:
features = features.drop(columns = ["duration","tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"])

### 1. Какие признаки имеют пропуски среди своих значений (приведите полный список имен этих признаков)?

In [5]:
print("В столбцах: \n")
for features_na in features:
    if features[features_na].isna().any() == True:
        print(features_na)
print("\n есть пропуски в значениях")

В столбцах: 

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time

 есть пропуски в значениях


Объяснение пустых значений.

#first_blood_time                True
Пустое значение, если первой крови не было в течении 5 минут. 
Буду делать задания по инструкции, но по хорошему заполнять эту графу нулями нечестно,
так как будет считаться, что первая кровь была сразу после начала игры и это будет влиять на качество.
Считаю, что эту колонку можно было бы дропнуть, так как первая кровь влияет на золото и опыт,
а они описаны в другиз столбцах.

#first_blood_team                True
Аналогично - первой крови не было в течении 5 минут, поэтому колонка пустая. 
#first_blood_player1             True
Первой крови не было - нет игрока, который её сделал
#first_blood_player2             True
#radiant_bottle_time             True
#radiant_courier_time            True
#radiant_flying_courier_time     True

все остальные пропуски - событие не произошло в течении первых 5 минут игры

In [6]:
#заполняем пропуски на 0
features = features.fillna(0)

### 2. Как называется столбец, содержащий целевую переменную? Столбец "radiant_win" - выиграла или проиграла команда

In [7]:
y = features.radiant_win

In [8]:
#обучающая выборка
X = features.drop(columns = 'radiant_win')

In [9]:
# 3.
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [10]:
import time

In [11]:
kf = KFold(n_splits=5, shuffle=True)

In [66]:
#качество градиентного бустинга для 10 деревьев и время выполнения кода
start_time = time.time()

clf = GradientBoostingClassifier(n_estimators = 10, random_state=0)
results = cross_val_score(clf, X, y, scoring="roc_auc", n_jobs=-1, cv=kf)
print(results.mean())

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

0.6648913490045358
--- 36.24032807350159 seconds ---


In [67]:
#качество градиентного бустинга для 20 деревьев и время выполнения кода
start_time = time.time()

clf = GradientBoostingClassifier(n_estimators = 20, random_state=0)
results = cross_val_score(clf, X, y, scoring="roc_auc", n_jobs=-1, cv=kf)
print(results.mean())

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

0.6822380245537951
--- 57.09689545631409 seconds ---


In [68]:
#качество градиентного бустинга для 30 деревьев и время выполнения кода
start_time = time.time()

clf = GradientBoostingClassifier(n_estimators = 30, random_state=0)
results = cross_val_score(clf, X, y,  scoring="roc_auc", n_jobs=-1, cv=kf)
print(results.mean())

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

0.6894520607374026
--- 87.09061670303345 seconds ---


In [70]:
#качество градиентного бустинга для 40 деревьев и время выполнения кода
start_time = time.time()

clf = GradientBoostingClassifier(n_estimators = 40, random_state=0)
results = cross_val_score(clf, X, y, scoring="roc_auc", n_jobs=-1, cv=kf)
print(results.mean())

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

0.6939327510259078
--- 55.26559138298035 seconds ---


### 3. Кросс-валидация для градиентного бустинга с 30 деревьями проводилась в течении 87 секунд. 
### Качество предсказаний равно примерно 0.69.



### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 
### Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Из отчета видно, что при повышении числа деревьев качество предсказаний повышается незначительно, а время выполнения
заметно увеличивается. Увеличение числа деревьев больше 30 не приведёт к значительным изменениям.
Чтобы ускорить обучение возможны следующие варианты:
1. Использование для обучения и кросс валидации не всю выборку, а только её подмножество.
2. Регулирование параметров, таких как: min_samples_split - Минимальное количество выборок, необходимых 
для разделения внутреннего узла; min_samples_leaf - Минимальное количество выборок, которое требуется 
для конечного узла; max_depth - Максимальная глубина дерева. И тд
3. Удаление ненужных столбцов, которые не должны влиять на результат. Например столбцы: 

start_time - время начала игры скорее всего не влияет на победу

first_blood_time - время первого убийства - считаю этот столбец ненужным, так как он не несёт 
в себе полезной информации для результата

first_blood_team - команда, которая совершила первое убийство. На первый взгляд важная колонка, но если разобраться,
то становится понятно, что первое убийство влияет на опыт(r1_xp) и на полученное золото (r1_gold)

first_blood_player1 - игрок, совершивший первое убийство - также считаю неважным столбцом, так как этот столбец связан
с опытом героя и золотом. А эти данные есть в других столбцах.

Таких столбцов достаточно много. Их исключение поможет скорости и качеству обучения.


# Подход 2: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [33]:
#Масштабируем признакм с помощью StandardScaler
from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()
scaler.fit(X)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [35]:
X_train = scaler.transform(X)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [36]:
X_train

array([[-2.54436416,  1.54068827, -1.24422828, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236, -0.92779756, -0.29225805, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  1.54068827, -0.5686365 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ...,
       [ 1.09874571, -0.57515673,  1.42743012, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.57515673,  1.48884755, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 ,  1.54068827, -0.04658831, ..., -0.55115386,
        -0.97042777, -0.79913812]])

In [ ]:
"""1. Какое качество получилось у логистической регрессии над всеми исходными признаками? 
Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? 
Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?"""

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
kf = KFold(n_splits=5, shuffle=True)

In [84]:
best_result = 0
best_c = 0
for c in range(-4, 6):
    clf = LogisticRegression(penalty = 'l2', C=10.0**c)
    results = cross_val_score(clf, X_train, y, scoring="roc_auc", n_jobs=-1, cv=kf)
    if best_result<results.mean():
        best_result = results.mean()
        best_c = 10.0 ** c
    print("Результат при С = ",10.0 ** c," равен ",results.mean())


print("\n Наилучший результат =", best_result, " при С = ", best_c)


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  0.0001  равен  0.7111614130197345


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  0.001  равен  0.7162907640322874


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

...

Результат при С =  1000.0  равен  0.7163426539698288


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  10000.0  равен  0.716444063220826


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  100000.0  равен  0.7164298286414372

 Наилучший результат = 0.7167140047446361  при С =  0.1


###  1. Наилучшее качество у логистической регрессии получилось равным  0.72, при этом параметр регуляризации C не сильно влияет на результат. У градиентного бустинга качество 0.7. Результат примерно одинаковый, но Логистическая регрессия работает значительно быстрее. Качество логистической регрессии немного лучше, так как входные данные были масштабированы

In [ ]:
"""  2. Как влияет на качество логистической регрессии удаление категориальных признаков 
(укажите новое значение метрики качества)? Чем можно объяснить это изменение?"""

In [44]:
#Удаляем категориальные признаки
X_drop_categ = X.drop(columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])

In [45]:
#Масштабируем признаки
scaler = StandardScaler()
scaler.fit(X_drop_categ)
X_drop_categ = scaler.transform(X_drop_categ)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [85]:
best_result = 0
best_c = 0
for c in range(-4, 6):
    clf = LogisticRegression(penalty = 'l2', C=10.0**c)
    results = cross_val_score(clf, X_drop_categ, y, scoring="roc_auc", n_jobs=-1, cv=kf)
    if best_result<results.mean():
        best_result = results.mean()
        best_c = 10.0**c
    print("Результат при С = ",10.0**c," равен ",results.mean())

print("\n Наилучший результат =", best_result, " при С = ", best_c)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  0.0001  равен  0.7112005717794405


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  0.001  равен  0.716357036113411


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

...

Результат при С =  1000.0  равен  0.7163906421323003


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  10000.0  равен  0.7163653037337323


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

Результат при С =  100000.0  равен  0.7164508989697731

 Наилучший результат = 0.7165180726321764  при С =  0.01


### 2. После удаления категориальных признаков качество логистической регрессии практически не изменилось = 0.72. Возможно это объясняется тем, что удалённые признаки имели очень маленькие веса и соответственно не влияли на конечный результат


### 3. 108 различных идентификаторов героев существует в данной игре.

In [48]:
# 108 различных идентификаторов героев
print(len((X.d1_hero.unique())))

108


In [49]:
# Проверим, что героев именно 108, несмотря на то, что наибольший идентификатор 112
sorted((X.r1_hero.unique()))==sorted((X.r2_hero.unique())) == sorted((X.r3_hero.unique()))==sorted((X.r4_hero.unique()))==sorted((X.r5_hero.unique()))==sorted((X.d1_hero.unique()))==sorted((X.d2_hero.unique()))==sorted((X.d3_hero.unique()))==sorted((X.d4_hero.unique()))==sorted((X.d5_hero.unique()))

True

In [50]:
# 4. Какое получилось качество при добавлении "мешка слов" по героям? 
# Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
import numpy as np

In [51]:
X_pick = np.zeros((X.shape[0], 112))

In [52]:
for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [53]:
# Добавим полученные признаки из мешка слов к числовым, которые использовали во втором пункте данного этапа.
X_4 = np.hstack((X_drop_categ, X_pick))

In [86]:
clf = LogisticRegression(penalty = 'l2', C=1)
results = cross_val_score(clf, X_4, y, scoring="roc_auc", n_jobs=-1, cv=kf)

results.mean()


C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

0.7518472602660764

### При добавлении мешка слов качество улучшилось и равно 0.75. Это объясняется тем, что новые признаки вносят больше позезной информации

In [ ]:
# 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
# Лучший алгоритм в нашем случае - Логистическая регрессия

In [55]:
#заполняем пропуски на 0
features_test = features_test.fillna(0)

In [56]:
#обработаем тестовые данные для лучшего предсказания. Удалим категориальные пирзнаки и масштабируем их
X_test = features_test.drop(columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])

In [57]:
scaler = StandardScaler()
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
# Добавим мешок слов
X_pick = np.zeros((features_test.shape[0], 112))

In [59]:
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [60]:
X_final_test = np.hstack((X_test_scaled, X_pick))

In [89]:
# Обучим нашу модель на тренировочных данных
clf = LogisticRegression(penalty = 'l2', C=1)
clf.fit(X_4, y)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
clf.predict_proba(X_final_test)[:, 1]

array([0.82566621, 0.75858951, 0.1870591 , ..., 0.23469622, 0.62844989,
       0.42558683])

### 5. Минимальное и максимальное значение прогноза на тестовой выборке

In [63]:
#Минимальное значение прогноза на тестовой выборке
clf.predict_proba(X_final_test)[:, 1].min()

0.008614713620377102

In [64]:
#Максимальное значение прогноза на тестовой выборке
clf.predict_proba(X_final_test)[:, 1].max()

0.9965116494292339